In [39]:
import torch
import string
import torch.nn as nn
from torch.nn import functional as F 


In [35]:
torch.backends.mps.is_available()

False

In [2]:
torch.cuda.is_available()

True

# Tokenizer

In [33]:
#abc_chars = string.ascii_lowercase
abc_chars = string.printable
abc_chars

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [36]:
stoi

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 'a': 10,
 'b': 11,
 'c': 12,
 'd': 13,
 'e': 14,
 'f': 15,
 'g': 16,
 'h': 17,
 'i': 18,
 'j': 19,
 'k': 20,
 'l': 21,
 'm': 22,
 'n': 23,
 'o': 24,
 'p': 25,
 'q': 26,
 'r': 27,
 's': 28,
 't': 29,
 'u': 30,
 'v': 31,
 'w': 32,
 'x': 33,
 'y': 34,
 'z': 35,
 'A': 36,
 'B': 37,
 'C': 38,
 'D': 39,
 'E': 40,
 'F': 41,
 'G': 42,
 'H': 43,
 'I': 44,
 'J': 45,
 'K': 46,
 'L': 47,
 'M': 48,
 'N': 49,
 'O': 50,
 'P': 51,
 'Q': 52,
 'R': 53,
 'S': 54,
 'T': 55,
 'U': 56,
 'V': 57,
 'W': 58,
 'X': 59,
 'Y': 60,
 'Z': 61,
 '!': 62,
 '"': 63,
 '#': 64,
 '$': 65,
 '%': 66,
 '&': 67,
 "'": 68,
 '(': 69,
 ')': 70,
 '*': 71,
 '+': 72,
 ',': 73,
 '-': 74,
 '.': 75,
 '/': 76,
 ':': 77,
 ';': 78,
 '<': 79,
 '=': 80,
 '>': 81,
 '?': 82,
 '@': 83,
 '[': 84,
 '\\': 85,
 ']': 86,
 '^': 87,
 '_': 88,
 '`': 89,
 '{': 90,
 '|': 91,
 '}': 92,
 '~': 93,
 ' ': 94,
 '\t': 95,
 '\n': 96,
 '\r': 97,
 '\x0b': 98,
 '\x0c': 99}

In [34]:
stoi = {ch: i for i, ch in enumerate(abc_chars)}
itos = {i:ch for i, ch in enumerate(abc_chars)}
encode = lambda s:[stoi[c] for c in s ] # str -> idx
decode = lambda i:''.join([itos[c] for c in i]) # idx -> str
print(encode("hello there"))
print(decode(encode("hello there")))

[17, 14, 21, 21, 24, 94, 29, 17, 14, 27, 14]
hello there


# Self-Attention

In [74]:
class SelfAttention(nn.Module):
    
    def __init__(self, n_dim_k : int, n_dim_v: int) -> None:
        super().__init__()
        self.n_dim_k = n_dim_k
        self.n_dim_v = n_dim_v
        self.keys = torch.randn(n_dim_k,n_dim_k)
        self.query = torch.randn(n_dim_k,n_dim_k)
        self.values = torch.randn(n_dim_v,n_dim_v)


    def forward(self, query, key ,value):

        mul_1 = torch.matmul(self.query, self.keys)
        scale = mul_1 / (self.n_dim_k**0.5)
        #print(out.shape)
        out = F.softmax(scale)
        print(out.shape)
        out = torch.matmul(out,self.values)

        return out


In [75]:
attention = SelfAttention(4,4)

In [76]:
attention()

TypeError: SelfAttention.forward() missing 3 required positional arguments: 'query', 'key', and 'value'

In [82]:
class MultiheadAttention(nn.Module):
        
    def __init__(self, n_heads: int, emb_dim : int, 
                 
                  ) -> None:
        

        super().__init__()

        self.n_dim_k = emb_dim//n_heads
        self.n_dim_v = emb_dim//n_heads
        self.n_heads = n_heads
        self.l_1 = nn.Linear(n_heads,emb_dim)
        self.l_2 = nn.Linear(int(n_heads*self.n_dim_v),emb_dim)

        self.keys = torch.randn(self.n_dim_k,self.n_dim_k)
        self.query = torch.randn(self.n_dim_k,self.n_dim_k)
        self.values = torch.randn(self.n_dim_v,self.n_dim_v)
        self.heads = nn.ModuleList(
            SelfAttention(self.n_dim_k, self.n_dim_v) for _ in range(n_heads)
        )

    def forward(self):

        project_keys = self.l_1(self.keys)
        project_query = self.l_1(self.query)
        project_values = self.l_1(self.values)

        out = self.heads( project_query,project_keys, project_values)

        out = torch.concat(out)

        out = self.l_2(out)




In [78]:
len(abc_chars)

100

In [83]:
multi_attention = MultiheadAttention(4,100)

In [84]:
multi_attention()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (25x25 and 4x100)

# Embeddings

# Encoder 

# Decoder